In [2]:
%load_ext autoreload
%autoreload 1
%aimport ecg_get_data
%aimport Models
%aimport train_test_validat
%aimport self_attention
%aimport ECGplot
%aimport Net
%aimport select_dataset
import select_dataset
import Models 
import Net
from train_test_validat import *
from self_attention import *
import  ecg_get_data 
import matplotlib.pyplot as plt
import ecg_plot

import torch
import torch.utils.data as Data
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import random

import time
import os
import gc

random_seed = 2
torch.manual_seed(random_seed)    # reproducible
torch.cuda.manual_seed_all(random_seed)
random.seed(random_seed)
np.random.seed(random_seed)

time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime()) 

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_path = './model/'+time_str
log_path = './log/'+  time_str
ECG_root = '/workspace/data/Preprocess_HTN/data/ECG'
EcgChannles_num = 12
EcgLength_num = 5000
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda:0


In [4]:
data = select_dataset.splite_dataset('/workspace/data/Preprocess_HTN/data/',True)

In [5]:
test_list = data.__get_test_file_list__(False)
test_Dataset = ecg_get_data.ECG_Dataset(ECG_root,test_list,EcgChannles_num,EcgLength_num)
# print(valid_list)
valid_list,train_list,addition_train_list = data.__get_VT_file_list__(0.3,True)
valid_Dataset = ecg_get_data.ECG_Dataset(ECG_root,valid_list,EcgChannles_num,EcgLength_num)
train_Dataset = ecg_get_data.ECG_Dataset(ECG_root,train_list,EcgChannles_num,EcgLength_num,ECG_root,addition_train_list)

	
 orginal   fliteryears
  33342       2484   
	
 orginal    fliterID 
   509        425    
	
 orginal    fliterID 
   2484       2272   
	
       HTN  NHTN 
test   425  2272 
npys:{%d} 2697
	
 orginal   fliteryears
  65933       8654   
	
 orginal    fliterID 
   1354       1221   
	
 orginal    fliterID 
   8654       8272   
	
       HTN  NHTN 
train  855   855 
test   366   366 
 add    0   7051 
npys:{%d} 732
npys:{%d} 1710
shadow_npys:{%d} 7051


In [6]:
ECG,label = test_Dataset.__getitem__(1)
#inf,path = train_Dataset.get_basic_inf(55)
#ecg_plot.plot(ECG*3500/1000, sample_rate = 500, title = "test",row_height= 10,show_grid=True,show_separate_line=True)
label
#ecg_plot.save_as_png(inf[1],'/workspace/data/OneDrive - mail.hfut.edu.cn/ECG/Interpretable_HTN//PNG_ECG/',dpi = 100)

tensor(1)

In [7]:
BATCH_SIZE = 64

FOLDS = 4
EPOCHS = 5000  
PATIENCE = 100
LR = 0.01


In [8]:
torch.cuda.empty_cache()# 清空显卡cuda
# NET = [
#     Net.channels_branch_CNN(True,res = True,se = True,Dropout_rate = 0.25)
# ]
NET = [Net.channels_branch_CNN(True,res = True,se = True,Dropout_rate = 0.25),Models.MLBFNet(1,12,0.01,True,True,True),Models.MLBFNet(1,12,0.1,True,True,True),Models.MLBFNet(1,12,0.2,True,True,True) ] # type: ignore

In [9]:
from torch.utils.tensorboard import SummaryWriter
os.makedirs(model_path, exist_ok=True)
writer = SummaryWriter(log_path)
writer.add_graph(NET[0], torch.zeros((1,12,5000)))  #模型及模型输入数据
torch.cuda.empty_cache()# 清空显卡cuda

In [10]:
import math
for fold in range(FOLDS):
    #每个人fold都重新抽取
    NET[fold].to(DEVICE)
    valid_list,train_list,addition_train_list = data.__get_VT_file_list__(0.3,True)
    valid_Dataset = ecg_get_data.ECG_Dataset(ECG_root,valid_list,EcgChannles_num,EcgLength_num)
    train_Dataset = ecg_get_data.ECG_Dataset(ECG_root,train_list,EcgChannles_num,EcgLength_num,ECG_root,addition_train_list)
    
    train_dataloader = Data.DataLoader(dataset=train_Dataset, batch_size=BATCH_SIZE, shuffle=True,num_workers=4,pin_memory=True)
    valid_dataloader = Data.DataLoader(dataset=valid_Dataset, batch_size=BATCH_SIZE, shuffle=True,num_workers=4,pin_memory=True)
    test_dataloader = Data.DataLoader(dataset=test_Dataset, batch_size=25, shuffle=True,num_workers=4,pin_memory=True)
    early_stopping = EarlyStopping(PATIENCE, verbose=True, model_path=model_path, delta=0, positive=False)
    optimizer  = torch.optim.Adam(NET[fold].parameters(), lr=LR,weight_decay=1e-2)  
    
    warm_up_iter = 5
    T_max = 40	# 周期
    lr_max = 0.01	# 最大值
    lr_min = 1e-5	# 最小值
    lambda0 = lambda cur_iter: cur_iter / warm_up_iter if  cur_iter < warm_up_iter else \
        (lr_min + 0.5*(lr_max-lr_min)*(1.0+math.cos( (cur_iter-warm_up_iter)/(T_max-warm_up_iter)*math.pi)))/0.1
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda0)
    criterion = torch.nn.CrossEntropyLoss()   
    # scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer=optimizer,base_lr=1e-5,max_lr=51e-3,step_size_down=20,step_size_up=20 ,cycle_momentum=False)
    #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 5, gamma=0.5)#等间隔调整学习率
    #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max = 24)
    best_loss = 3
    for epoch in range(1,EPOCHS):
        time_all=0
        start_time = time.time()
        train_loss,train_acc = train_model(train_dataloader, NET[fold], criterion, optimizer,DEVICE) # 训练模型
        time_all = time.time()-start_time
        
        y_true,y_pred,validate_loss,validate_acc = eval_model(valid_dataloader,criterion,NET[fold],DEVICE) # 验证模型
        y_true_test,y_pred_test,loss_test,acc_test = eval_model(test_dataloader,criterion,NET[fold],DEVICE) # 测试模型
        F1_score =f1_score(y_true, y_pred, average='macro')#F1分数
        F1_score_test =f1_score(y_true_test, y_pred_test, average='macro')#F1分数
        writer.add_scalars(main_tag=str(fold)+'_Loss',tag_scalar_dict={'train': train_loss,'validate': validate_loss,'test': loss_test},global_step=epoch)
        writer.add_scalars(main_tag=str(fold)+'_Accuracy',tag_scalar_dict={'train': train_acc,'validate': validate_acc,'test': acc_test},global_step=epoch)
        writer.add_scalars(main_tag=str(fold)+'_LearningRate',tag_scalar_dict={'LR': optimizer.state_dict()['param_groups'][0]['lr']},global_step=epoch)
        writer.add_scalars(main_tag=str(fold)+'_F1_score',tag_scalar_dict={'validate': F1_score,'test':F1_score_test},global_step=epoch)
        print('- Epoch: %d - Train_loss: %.5f - Train_acc: %.5f - Val_loss: %.5f - Val_acc: %.5f - F1 score: %.5f - T_Time: %.5f' %(epoch,train_loss,train_acc,validate_loss,validate_acc,F1_score,time_all))
        print('当前学习率：%.8f' %optimizer.state_dict()['param_groups'][0]['lr'])
        C1 = confusion_matrix(y_true,y_pred)
        C2 = confusion_matrix(y_true_test,y_pred_test)
        print('validate:\n',C1)
        print('test:\n' ,C2)
        if validate_loss < best_loss:
            best_loss = validate_loss
            print('Find better model in Epoch {0}, saving model.'.format(epoch))
            #torch.save(NET[fold].state_dict(), model_path+'/parameter_best_model_' + str(fold) + '.pt')
        if C2[1,1]>300 and C2[0,0]>1500 :
            torch.save(NET[fold].state_dict(), model_path+'/parameter_test_best_model_' + str(fold) + '.pt')
        
        scheduler.step() # 学习率迭代
        #是否满足早停法条件
        if(early_stopping(validate_loss,NET[fold],fold)):
            print("Early stopping")
            break

    print('Fold %d Training Finished' %(fold+1))
    torch.cuda.empty_cache()# 清空显卡cuda
print('Training Finished')

	
 orginal   fliteryears
  65933       8654   
	
 orginal    fliterID 
   1354       1221   
	
 orginal    fliterID 
   8654       8272   
	
       HTN  NHTN 
train  855   855 
test   366   366 
 add    0   7051 
npys:{%d} 732
npys:{%d} 1710
shadow_npys:{%d} 7051
- Epoch: 1 - Train_loss: 0.69646 - Train_acc: 0.50659 - Val_loss: 0.69495 - Val_acc: 0.49833 - F1 score: 0.33333 - T_Time: 35.31313
当前学习率：0.00000000
validate:
 [[  0 366]
 [  0 366]]
test:
 [[   0 2272]
 [   0  425]]
Find better model in Epoch 1, saving model.
Validation F1 score  increase to (inf --> -0.69494510).  Saving model ...
                    --------------------------------------------------

- Epoch: 2 - Train_loss: 0.61459 - Train_acc: 0.67575 - Val_loss: 0.82089 - Val_acc: 0.49163 - F1 score: 0.33333 - T_Time: 32.24880
当前学习率：0.00200000
validate:
 [[366   0]
 [366   0]]
test:
 [[2256   16]
 [ 425    0]]
EarlyStopping counter: 1 out of 100

- Epoch: 3 - Train_loss: 0.59476 - Train_acc: 0.69673 - Val_loss: 0.70988 -

KeyboardInterrupt: 